***Prvi deo***

In [1]:
import numpy as np
from numpy import linalg as LA

In [2]:
def ParametriKamere(T):

    T0 = T[:, :-1]

    if LA.det(T0) < 0:
        T = -T

    # -- C --------------------------------------------
    c1 = LA.det(T[:, 1:])
    c2 = -LA.det(np.array([T[:,0], T[:,2], T[:,3]]).T)
    c3 = LA.det(np.array([T[:,0], T[:,1], T[:,3]]).T)
    c4 = -LA.det(T[:, :-1])

    c1 = c1/c4
    c2 = c2/c4
    c3 = c3/c4

    C = np.array([c1, c2, c3])

    # -------------------------------------------------

    Q, R = LA.qr(LA.inv(T0))

    if R[0][0] < 0:
        R[0] = -R[0]
        Q[:,0] = -Q[:,0]
    if R[1][1] < 0:
        R[1] = -R[1]
        Q[:,1] = -Q[:,1]
    if R[2][2] < 0:
        R[2] = -R[2]
        Q[:,2] = -Q[:,2]

    # A 
    A = Q.T

    # K = R^-1
    K = T0.dot(Q)

    if K[2][2] != 1:
        K = K / K[2][2]

    return K, A, C

In [3]:
n = 1

In [4]:
T = np.array([[5, -1-2*n, 3, 18-3*n],
             [0, -1, 5, 21],
             [0, -1, 0, 1]])

In [5]:
K, A, C = ParametriKamere(T)
print("K:\n", K.round(5))
print("A:\n", A.round(5))
print("C:\n", C.round(5))

K:
 [[5. 3. 3.]
 [0. 5. 1.]
 [0. 0. 1.]]
A:
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]]
C:
 [-0.  1. -4.]


***Drugi deo***

In [6]:
M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50*n, 1])
M5 = np.array([30*n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])

In [7]:
M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])

In [8]:
originali = np.array([M1, M2, M3, M4, M5, M6])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

In [9]:
def CameraDLP(originali, projekcije):
    x = originali[0][0]
    y = originali[0][1]
    z = originali[0][2]
    t = originali[0][3]

    u = projekcije[0][0]
    v = projekcije[0][1]
    w = projekcije[0][2]

    A = np.array([
        [0, 0, 0, 0, -w*x, -w*y, -w*z, -w*t, v*x, v*y, v*z, v*t],
        [w*x, w*y, w*z, w*t, 0, 0, 0, 0, -u*x, -u*y, -u*z, -u*t]
    ])

    for i in range(1, len(originali)):
        x = originali[i][0]
        y = originali[i][1]
        z = originali[i][2]
        t = originali[i][3]

        u = projekcije[i][0]
        v = projekcije[i][1]
        w = projekcije[i][2]

        row1 = np.array([0, 0, 0, 0, -w*x, -w*y, -w*z, -w*t, v*x, v*y, v*z, v*t])
        row2 = np.array([w*x, w*y, w*z, w*t, 0, 0, 0, 0, -u*x, -u*y, -u*z, -u*t])

        A = np.vstack((A, row1))
        A = np.vstack((A, row2))

    # SVD dekompozicija
    U, S, V = np.linalg.svd(A)

    T = V[-1].reshape(3, 4)
    
    return T

In [10]:
T = CameraDLP(originali, projekcije)
T = T / T[0][0]
print("T:\n", T)

T:
 [[ 1.00000000e+00  7.44887603e-02 -5.06787057e+00  2.01818651e+03]
 [-1.96113036e+00  3.04808672e+00 -9.94850183e-01  1.37344176e+03]
 [-1.08348240e-03 -2.55445225e-03 -3.78543549e-03  6.43992929e+00]]
